In [1]:
from pyspark.sql.functions import col

from src.dependencies.constant import TRAIN_PATH, TEST_PATH, CORPUS_PATH
from src.dependencies.spark import SparkIRSystem
from src.preprocessing.data_loader import load_data

In [2]:
spark = SparkIRSystem()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/08 17:58:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
train_data = load_data(spark, TRAIN_PATH)

In [4]:
test_data = load_data(spark, TEST_PATH)

In [5]:
corpus_data = load_data(spark, CORPUS_PATH)

In [31]:
corpus_data.columns

['text', 'cid']

In [33]:
corpus_data.count(), corpus_data.select("cid").distinct().count(), corpus_data.select("text").distinct().count()

(261597, 261597, 261595)

In [4]:
train_data.columns

['question', 'context_list', 'qid', 'cid']

In [34]:
test_data.show(1)

+--------------------+--------------------+-----+--------+
|            question|        context_list|  qid|     cid|
+--------------------+--------------------+-----+--------+
|Phó Tổng Giám đốc...|[Áp dụng chế độ t...|70867|[140864]|
+--------------------+--------------------+-----+--------+
only showing top 1 row


In [38]:
corpus_data.filter("cid = 140864").show()

+--------------------+------+
|                text|   cid|
+--------------------+------+
|Áp dụng chế độ ti...|140864|
+--------------------+------+



In [12]:
train_data.describe()

DataFrame[summary: string, question: string, qid: string]

In [18]:
corpus.printSchema()

root
 |-- text: string (nullable = true)
 |-- cid: long (nullable = true)



In [22]:
train_data.show(n = 1, truncate=False)

+-------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+--------+
|question                                                                                          

In [23]:
from pyspark.sql import functions as F

In [25]:
qa = train_data.withColumn("context", F.explode("context_list")).select("qid", "cid", "question", "context")
qa.show(5)

+------+--------------+--------------------+--------------------+
|   qid|           cid|            question|             context|
+------+--------------+--------------------+--------------------+
| 72600|      [142820]|Liên đoàn Luật sư...|“Điều 2. Địa vị p...|
|147562|[27817, 72117]|Tên hợp tác xã bị...|"Điều 7. Tên hợp ...|
|147562|[27817, 72117]|Tên hợp tác xã bị...|Cơ quan đăng ký h...|
|142107|[33215, 56201]|Tài xế lái xe ô t...|"1. Sử dụng lái x...|
|142107|[33215, 56201]|Tài xế lái xe ô t...|“Điều 21. Khám sứ...|
+------+--------------+--------------------+--------------------+
only showing top 5 rows


In [26]:
corpus = qa.select("cid","context").dropDuplicates(["cid"])

In [6]:
examples = {'train': [], 'test': []}

In [9]:
from pyspark.sql.functions import explode
exploded = train_data.select(col("qid"), col("question"), explode(col("context_list")).alias("context"))
exploded.show(10)

+------+--------------------+--------------------+
|   qid|            question|             context|
+------+--------------------+--------------------+
| 72600|Liên đoàn Luật sư...|“Điều 2. Địa vị p...|
|147562|Tên hợp tác xã bị...|"Điều 7. Tên hợp ...|
|147562|Tên hợp tác xã bị...|Cơ quan đăng ký h...|
|142107|Tài xế lái xe ô t...|"1. Sử dụng lái x...|
|142107|Tài xế lái xe ô t...|“Điều 21. Khám sứ...|
| 77353|Các bước chuẩn bị...|BỘT CRAVATE\n...\...|
|113090|Viên chức Hộ sinh...|Hộ sinh hạng IV -...|
| 24619|Nơi tạm trú được ...|8. Nơi thường trú...|
|139884|Sỹ quan boong tàu...|Điều 6. Tiêu chuẩ...|
|139884|Sỹ quan boong tàu...|Điều 6. Tiêu chuẩ...|
+------+--------------------+--------------------+
only showing top 10 rows


In [11]:
exploded = train_data.select(col("question").alias("text_0"),
                             explode(col("context_list")).alias("text_1"))

In [12]:
exploded.show(10)

+--------------------+--------------------+
|              text_0|              text_1|
+--------------------+--------------------+
|Liên đoàn Luật sư...|“Điều 2. Địa vị p...|
|Tên hợp tác xã bị...|"Điều 7. Tên hợp ...|
|Tên hợp tác xã bị...|Cơ quan đăng ký h...|
|Tài xế lái xe ô t...|"1. Sử dụng lái x...|
|Tài xế lái xe ô t...|“Điều 21. Khám sứ...|
|Các bước chuẩn bị...|BỘT CRAVATE\n...\...|
|Viên chức Hộ sinh...|Hộ sinh hạng IV -...|
|Nơi tạm trú được ...|8. Nơi thường trú...|
|Sỹ quan boong tàu...|Điều 6. Tiêu chuẩ...|
|Sỹ quan boong tàu...|Điều 6. Tiêu chuẩ...|
+--------------------+--------------------+
only showing top 10 rows


In [13]:
df = train_data.select(col("qid"), col("question"), explode(col("cid")).alias("cid"))
queries = {}
relevant_docs = {}
for row in df.toLocalIterator():
    qid = row.qid
    qid_str = f"q{qid}"
    cid_str = f"c{row.cid}"

    queries[qid_str] = row.question

    if qid_str not in relevant_docs:
        relevant_docs[qid_str] = {}

    relevant_docs[qid_str][cid_str] = 1

In [16]:
queries

{'q72600': 'Liên đoàn Luật sư Việt Nam là tổ chức xã hội – nghề nghiệp có tư cách pháp nhân, có con dấu, tài khoản riêng?',
 'q147562': 'Tên hợp tác xã bị rơi vào trường hợp cấm thì cơ quan nào có quyền từ chối chấp thuận đối với tên đó?',
 'q142107': 'Tài xế lái xe ô tô khách 50 chỗ ngồi bao lâu thì doanh nghiệp phải tổ chức khám sức khỏe định kỳ 1 lần?',
 'q77353': 'Các bước chuẩn bị thủ thuật bó bột Cravate sẽ như thế nào?',
 'q113090': 'Viên chức Hộ sinh hạng 4 có những nhiệm vụ gì trong công tác chăm sóc sức khỏe sinh sản cộng đồng?',
 'q24619': 'Nơi tạm trú được quy định như thế nào?',
 'q139884': 'Sỹ quan boong tàu từ 500 GT trở lên của tàu biển Việt Nam cần đáp ứng những tiêu chuẩn chuyên môn gì?',
 'q157240': 'Mục đích của giám sát của Mặt trận Tổ quốc Việt Nam là gì?',
 'q136457': 'Khi tham gia bảo hiểm bắt buộc trách nhiệm dân sự cần tuân theo những nguyên tắc gì?',
 'q147796': 'Hết thời gian tạm ngừng hoạt động mà công ty chứng khoán chưa hoạt động lại thì có bị thu hồi Giấ

In [17]:
relevant_docs

{'q72600': {'c142820': 1},
 'q147562': {'c27817': 1, 'c72117': 1},
 'q142107': {'c33215': 1, 'c56201': 1},
 'q77353': {'c148158': 1},
 'q113090': {'c188132': 1},
 'q24619': {'c68975': 1},
 'q139884': {'c3720': 1},
 'q157240': {'c88914': 1},
 'q136457': {'c66253': 1},
 'q147796': {'c48911': 1},
 'q128081': {'c62866': 1},
 'q146034': {'c224996': 1},
 'q76434': {'c70462': 1},
 'q19651': {'c1536': 1},
 'q166407': {'c247802': 1},
 'q44619': {'c17986': 1},
 'q77830': {'c143439': 1},
 'q4299': {'c66240': 1, 'c48853': 1},
 'q80318': {'c105941': 1},
 'q43343': {'c110013': 1},
 'q99436': {'c104389': 1},
 'q73069': {'c4637': 1},
 'q40518': {'c72252': 1},
 'q29815': {'c47569': 1},
 'q164548': {'c245691': 1},
 'q71112': {'c62695': 1},
 'q166932': {'c70954': 1},
 'q69349': {'c3159': 1},
 'q113308': {'c63703': 1},
 'q99012': {'c172421': 1},
 'q95422': {'c46218': 1},
 'q142016': {'c154541': 1, 'c130493': 1},
 'q88080': {'c63353': 1},
 'q38523': {'c104619': 1},
 'q114057': {'c173319': 1},
 'q13282': {'